In [28]:
import requests
from datetime import datetime, timezone

# 1️⃣ Use today's UTC date in the filename
today_str = datetime.now(timezone.utc).strftime("%Y%m%d")
tle_file = f'tle_data_{today_str}.txt'

# 2️⃣ CelesTrak's TLE for "Active Satellites" (updated and working)
url = 'https://celestrak.com/NORAD/elements/gp.php?GROUP=active&FORMAT=tle'

# 3️⃣ Fetch and save TLE data
response = requests.get(url)
if response.status_code == 200:
    with open(tle_file, 'w', encoding='utf-8') as f:
        f.write(response.text)
    print(f"✅ TLE data downloaded and saved to: {tle_file}")
else:
    print(f"❌ Failed to download TLE data. Status code: {response.status_code}")


✅ TLE data downloaded and saved to: tle_data_20250605.txt


In [29]:
# Step 2: Load saved TLE file and compute satellite geolocations

from skyfield.api import load, wgs84
from datetime import datetime, timezone
import pandas as pd
import os

# 1️⃣ Generate today's TLE file name
today_str = datetime.now(timezone.utc).strftime("%Y%m%d")
tle_file = f'tle_data_{today_str}.txt'

# 2️⃣ Check if the TLE file exists
if not os.path.exists(tle_file):
    raise FileNotFoundError(f"TLE file {tle_file} not found. Please run Step 1 first.")

# 3️⃣ Load satellites from saved TLE file
satellites = load.tle_file(tle_file)
print(f"📦 Loaded {len(satellites)} satellites")

# 4️⃣ Get current UTC time as Skyfield time object
ts = load.timescale()
now = ts.now()

# 5️⃣ Compute ground subpoints for first N satellites
records = []
for sat in satellites[:20]:  # Limit to first 20 satellites for testing
    try:
        geocentric = sat.at(now)
        subpoint = wgs84.subpoint(geocentric)

        records.append({
            'name': sat.name,
            'latitude': subpoint.latitude.degrees,
            'longitude': subpoint.longitude.degrees,
            'altitude_km': subpoint.elevation.km,
            'dist_from_equator': abs(subpoint.latitude.degrees)
        })
    except Exception as e:
        print(f"⚠️ Skipping {sat.name} due to error: {e}")

# 6️⃣ Create DataFrame
df = pd.DataFrame(records)

# 7️⃣ Print sample data
print("📊 Sample satellite data:")
print(df.head())


📦 Loaded 11732 satellites
📊 Sample satellite data:
           name   latitude   longitude  altitude_km  dist_from_equator
0   CALSPHERE 1 -83.747359   -0.516784  1003.768008          83.747359
1   CALSPHERE 2 -48.532990    1.613339  1077.041355          48.532990
2         LCS 1  -4.768434  113.448855  2778.711610           4.768434
3     TEMPSAT 1  12.705993  147.350984  1178.591702          12.705993
4  CALSPHERE 4A -47.269182   60.093910  1178.815662          47.269182


📊 Sample satellite data:


,name,latitude,longitude,altitude_km,dist_from_equator
0,CALSPHERE 1,-80.740636,-1.384338,1002.439366,80.740636
1,CALSPHERE 2,-45.558188,1.366582,1076.444187,45.558188
2,LCS 1,-5.923180,115.093033,2778.653851,5.923180
3,TEMPSAT 1,15.623822,147.131364,1180.025650,15.623822
4,CALSPHERE 4A,-44.360758,59.881268,1175.563249,44.360758


In [34]:
# Step 3: Compute Satellite Position Features (Lat, Lon, Altitude)

from skyfield.api import wgs84
from datetime import datetime, timezone

def compute_satellite_features(satellites, ts):
    """
    Computes geodetic features (lat, lon, altitude) for each satellite.
    Parameters:
        satellites: list of skyfield EarthSatellite objects
        ts: Skyfield TimeScale object
    Returns:
        DataFrame with satellite name, latitude, longitude, altitude_km, distance from equator
    """
    now = ts.now()  # ✅ Must use Skyfield Time object
    data = []

    for sat in satellites:
        try:
            geocentric = sat.at(now)  # This requires a Time object
            subpoint = wgs84.subpoint(geocentric)
            lat = subpoint.latitude.degrees
            lon = subpoint.longitude.degrees
            alt = subpoint.elevation.km
            dist_from_equator = abs(lat)

            data.append({
                'name': sat.name,
                'latitude': lat,
                'longitude': lon,
                'altitude_km': alt,
                'dist_from_equator': dist_from_equator
            })
        except Exception as e:
            print(f"⚠️ Failed for {sat.name}: {e}")

    df = pd.DataFrame(data)
    return df

# ✅ Usage Example:
df_features = compute_satellite_features(satellites, ts)
print("📊 Sample satellite features:")
print(df_features.head())


📊 Sample satellite features:
           name   latitude   longitude  altitude_km  dist_from_equator
0   CALSPHERE 1   6.830866   -9.095245   960.968640           6.830866
1   CALSPHERE 2  40.409051   -5.426884  1079.377442          40.409051
2         LCS 1 -30.881405  170.598026  2785.118721          30.881405
3     TEMPSAT 1  80.869062  -39.376696  1176.848692          80.869062
4  CALSPHERE 4A  40.714128   53.664907  1091.683337          40.714128


In [35]:
# Step 4: Classify Risk Levels Based on Satellite Features

def classify_risk_levels(df):
    """
    Adds a 'risk_level' column based on distance from equator.
    Thresholds:
        - Low: <= 10 degrees
        - Medium: 10–50 degrees
        - High: > 50 degrees
    Parameters:
        df: DataFrame with satellite features (including 'dist_from_equator')
    Returns:
        Updated DataFrame with a new 'risk_level' column
    """
    def risk_category(dist):
        if dist <= 10:
            return 'Low'
        elif dist <= 50:
            return 'Medium'
        else:
            return 'High'

    df['risk_level'] = df['dist_from_equator'].apply(risk_category)
    return df

# ✅ Apply risk classification to satellite features
df_features = classify_risk_levels(df_features)

print("🛰️ Satellite features with risk levels:")
print(df_features[['name', 'latitude', 'longitude', 'altitude_km', 'risk_level']].head())


🛰️ Satellite features with risk levels:
           name   latitude   longitude  altitude_km risk_level
0   CALSPHERE 1   6.830866   -9.095245   960.968640        Low
1   CALSPHERE 2  40.409051   -5.426884  1079.377442     Medium
2         LCS 1 -30.881405  170.598026  2785.118721     Medium
3     TEMPSAT 1  80.869062  -39.376696  1176.848692       High
4  CALSPHERE 4A  40.714128   53.664907  1091.683337     Medium


### 🚨 Risk Level Classification Based on Distance from Equator

This section assigns a `risk_level` to each satellite using its distance from the equator (in degrees of latitude). This is used as a proxy to assess potential conjunction risk or exposure to debris and traffic.

**Thresholds:**
- **Low Risk** (≤ 10°): Equatorial or near-equatorial orbits. Lower orbital traffic.
- **Medium Risk** (10°–50°): Moderately inclined orbits. Moderate intersection probability.
- **High Risk** (> 50°): Polar or sun-synchronous orbits. Highest chance of orbital crossings.

> 🔍 Note: This is a basic heuristic. It can later be improved by integrating actual debris maps, relative velocities, or orbital crossings.


In [36]:
# Step 5: Plot Satellite Subpoints and Risk Levels Using Plotly

import plotly.graph_objects as go

def plot_satellite_risk_map(df, sample_size=100):
    """
    Plots an interactive map of satellites with color-coded risk levels.
    Parameters:
        df: DataFrame with satellite 'latitude', 'longitude', 'name', and 'risk_level'
        sample_size: Number of satellites to display (for readability)
    """
    # ✅ Subset data for readability
    df_sample = df.sample(n=min(sample_size, len(df)), random_state=42)

    # Color map for risk levels
    color_map = {'Low': 'green', 'Medium': 'orange', 'High': 'red'}
    df_sample['color'] = df_sample['risk_level'].map(color_map)

    # Create interactive globe plot
    fig = go.Figure(go.Scattergeo(
        lon = df_sample['longitude'],
        lat = df_sample['latitude'],
        text = df_sample['name'] + '<br>Risk: ' + df_sample['risk_level'],
        mode = 'markers+text',
        marker = dict(
            size = 6,
            color = df_sample['color'],
            line = dict(width=0.5, color='black'),
            opacity = 0.8
        ),
        textfont = dict(size=8, color='black'),
        textposition = 'top center'
    ))

    # Set layout for the map
    fig.update_layout(
        title='🛰️ Satellite Risk Levels Around the Globe',
        geo=dict(
            projection_type='natural earth',
            showland=True,
            landcolor='rgb(230, 230, 230)',
            showocean=True,
            oceancolor='rgb(200, 230, 255)',
            coastlinecolor='gray'
        ),
        margin=dict(l=0, r=0, t=30, b=0)
    )

    fig.show()

# 🔍 Run the visualization
plot_satellite_risk_map(df_features, sample_size=50)
